<a href="https://colab.research.google.com/github/tyukios/genai/blob/main/trpg_new1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -fsSL https://ollama.ai/install.sh | sh
!nohup ollama serve &
!ollama pull gemma3:4b
!pip install openai gradio

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
nohup: appending output to 'nohup.out'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.2 MB/s eta 0:00:00


In [2]:
import openai
from openai import OpenAI
api_key = "ollama"
client = OpenAI(
    api_key=api_key,
    base_url="http://localhost:11434/v1"
)

In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
使用Ollama (OpenAI兼容API) 的AI互動式故事生成器
"""

import gradio as gr
import os
import json
import time
from datetime import datetime
import random
from openai import OpenAI
from typing import List, Dict, Any, Tuple

class StoryLLM:
    def __init__(self):
        # 初始化Ollama客户端
        self.client = OpenAI(
            api_key="ollama",  # 可以是任意字符串
            base_url="http://localhost:11434/v1"
        )

        # 可用模型列表（需要预先用ollama pull下载）
        self.available_models = ["gemma3:4b"]
        self.current_model = "gemma3:4b"


        # 各類型的預設提示模板
        self.genre_prompts = {
            "奇幻": "你是一位資深奇幻作家，擅長創作充滿魔法、龍與古老預言的故事。",
            "科幻": "你是一位科幻小說專家，精通硬科幻和科技細節描寫。",
            "懸疑": "你是一位懸疑大師，擅長編織複雜的謎題和出人意料的轉折。",
            "愛情": "你是一位浪漫小說作家，擅長描寫細膩情感和人際關係。",
            "冒險": "你是一位冒險故事專家，擅長描寫刺激的動作場面和異國風情。",
            "歷史": "你是一位歷史小說作家，擅長將真實歷史與虛構情節完美結合。",
            "恐怖": "你是一位恐怖大師，擅長營造心理恐懼和超自然氛圍。"
        }

    def generate_with_ollama(self, prompt: str, system_prompt: str = None) -> str:
        """使用Ollama生成內容"""
        messages = []

        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})

        messages.append({"role": "user", "content": prompt})

        try:
            response = self.client.chat.completions.create(
                model=self.current_model,
                messages=messages,
                temperature=0.7,
                max_tokens=2000
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"Ollama生成失敗: {str(e)}")
            return f"錯誤: 故事生成失敗 ({str(e)})"

    def generate_story_start(self, genre: str, custom_prompt: str = "") -> Dict[str, Any]:
        """生成故事開頭"""
        # 構建system prompt
        system_prompt = self.genre_prompts.get(genre, "")
        if custom_prompt:
            system_prompt += f"\n特別要求: {custom_prompt}"

        # 構建用戶提示
        user_prompt = f"""請用繁體中文創作一個{genre}故事的開頭，包含:
1. 引人入勝的開場（使用Markdown二級標題）
2. 主角簡介（職業/特質）
3. 故事發生的獨特場景
4. 初始衝突或目標的暗示

最後提供3個讀者可以選擇的行動選項，格式為：
選項1: [描述]
選項2: [描述]
選項3: [描述]"""

        # 獲取LLM回應
        full_response = self.generate_with_ollama(user_prompt, system_prompt)

        # 分離故事內容和選項
        story_text = "\n".join([line for line in full_response.split("\n") if not line.startswith("選項")])
        options = [line.split(": ")[1] for line in full_response.split("\n") if line.startswith("選項")]

        if not options:  # 如果LLM沒有生成選項，使用默認
            options = [
                "探索周圍環境，尋找線索",
                "與附近的角色交談",
                "檢查你的裝備和資源"
            ]

        return {
            "genre": genre,
            "system_prompt": system_prompt,
            "story_text": story_text,
            "options": options
        }

    def continue_story(self, history: List[Dict], user_input: str) -> Dict[str, Any]:
        """繼續故事"""
        if not history:
            return self.generate_story_start("奇幻")  # 默認回退

        last_part = history[-1]
        genre = last_part.get("genre", "奇幻")

        # 構建繼續提示
        prompt = f"""繼續這個{genre}故事:

已發生的故事:
{last_part['story_text']}

讀者選擇:
{user_input}

請:
1. 根據選擇發展合理情節（使用Markdown二級標題）
2. 保持風格一致
3. 創造3個新選項（格式：選項1: [描述]）
4. 適當加入對話和場景描寫"""

        # 獲取LLM回應
        full_response = self.generate_with_ollama(
            prompt,
            system_prompt=last_part.get("system_prompt", "")
        )

        # 分離故事內容和選項
        story_text = "\n".join([line for line in full_response.split("\n") if not line.startswith("選項")])
        options = [line.split(": ")[1] for line in full_response.split("\n") if line.startswith("選項")]

        if not options:  # 後備選項
            options = [
                "繼續探索當前區域",
                "與新出現的角色互動",
                "嘗試解決當前困境"
            ]

        return {
            "genre": genre,
            "story_text": story_text,
            "options": options
        }

# Gradio界面（與之前版本類似，但增加模型選擇）
def create_gradio_interface():
    story_gen = StoryLLM()

    with gr.Blocks(title="Ollama故事生成器 (OpenAI API格式)") as demo:
        # 模型選擇
        with gr.Row():
            model_selector = gr.Dropdown(
                story_gen.available_models,
                value=story_gen.current_model,
                label="選擇Ollama模型"
            )

            def update_model(model):
                story_gen.current_model = model
                return f"已切換模型: {model}"

            model_selector.change(update_model, model_selector, gr.Textbox())

        # 故事設置
        genre = gr.Dropdown(
            list(story_gen.genre_prompts.keys()),
            value="奇幻",
            label="故事類型"
        )
        custom_prompt = gr.Textbox(
            label="客製化要求 (可選)",
            placeholder="例如：加入時間旅行元素/主角是反英雄..."
        )
        start_btn = gr.Button("開始故事", variant="primary")

        # 故事顯示
        story_display = gr.Markdown(label="故事發展")
        options_radio = gr.Radio(label="你的選擇", interactive=True)
        continue_btn = gr.Button("繼續故事", variant="primary")

        # 保存功能
        save_btn = gr.Button("保存故事")
        save_output = gr.Textbox(label="保存狀態", interactive=False)

        # 狀態存儲
        current_story = gr.State([])

        # 事件處理
        def start_story(genre, custom_prompt, model):
            story_gen.current_model = model
            story = story_gen.generate_story_start(genre, custom_prompt)
            return (
                story['story_text'],
                gr.Radio(choices=story['options'], value=None),
                [story]  # 存入history
            )

        def continue_story(history, choice):
            if not choice:
                return "請先選擇一個行動！", gr.Radio(), history

            new_part = story_gen.continue_story(history, choice)
            updated_history = history + [new_part]
            return (
                new_part['story_text'],
                gr.Radio(choices=new_part['options'], value=None),
                updated_history
            )

        def save_story(history):
            if not history:
                return "沒有故事可保存"

            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"story_{timestamp}.json"

            os.makedirs("stories", exist_ok=True)
            with open(f"stories/{filename}", "w", encoding="utf-8") as f:
                json.dump(history, f, ensure_ascii=False, indent=2)

            return f"故事已保存為: stories/{filename}"

        start_btn.click(
            start_story,
            inputs=[genre, custom_prompt, model_selector],
            outputs=[story_display, options_radio, current_story]
        )

        continue_btn.click(
            continue_story,
            inputs=[current_story, options_radio],
            outputs=[story_display, options_radio, current_story]
        )

        save_btn.click(
            save_story,
            inputs=[current_story],
            outputs=[save_output]
        )

    return demo

if __name__ == "__main__":
    # 檢查Ollama服務是否可用
    try:
        test_client = OpenAI(
            api_key="ollama",
            base_url="http://localhost:11434/v1"
        )
        test_client.models.list()  # 測試連接
    except Exception as e:
        print("無法連接Ollama服務，請確保：")
        print("1. 已安裝Ollama: curl -fsSL https://ollama.ai/install.sh | sh")
        print("2. 已啟動服務: ollama serve")
        print("3. 已下載至少一個模型，如: ollama pull gemma3:4b")
        exit(1)

    demo = create_gradio_interface()
    demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://88ecffa7bc19175ea2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
